# 4.	Time Series Analysis for Prediction
The data extracted to perform the modeling was for the period following the New Upgrade launched in December 2019. The selection of the said period was due to the impact of the New Upgrade. The efficiency of the New Upgrade had improved the number of issues logged and resolution time, as portrayed in Section 3. As a result, modeling done before the new upgrade was launched will be inaccurate.
However, the limitation to the selected period was a short duration of 17 months, where it may not contain sufficient data to develop a robust model. Thus, we have split our dataset into two parts to validate the model; a training dataset extracted from the period December 2019 to January 2021 (14 months) and test dataset from February to April 2021 (3 months). The objective is on the prediction for the following 3 months.
<br><br>
## 4.2.	Time Series Modeling – All Countries
On our dataset, multiple time series modeling was performed in order to attain the best fit time series model for prediction. The majority of the model did not perform well in terms of issue prediction. The complete list may be found in Appendix 1 and Appendix 2, where the graphs in Appendix 2 illustrate how the predicted values were unable to fit our test values.
<br><br>


### 4.2.1.	Exponential Smoothing – Holt’s
We ran Holt’s Smoothing model on a monthly basis for the training dataset, with a prediction to fit the test dataset. The graph illustrates the prediction line in blue, while the actual test dataset was in red. The predicted value and trend of the slope are different compared to our test dataset. Hence, the model from Holt’s smoothing method is weak for prediction.

In [ ]:
```{r Time Series - All Country - Exponential Smoothing - Holts}
# Train the time series model by using exponential smoothing - Holt's and Holt's Winter method

# Data by month:
# plot the data to check the trend and seasonality
total.issue=month.2019$total
plot(total.issue,type = "o")
# there is trend component as shown in the graph 

<img src="https://drive.google.com/uc?export=view&id=143_vKv6sqspnPT-5QYHU8JvULoftGxni" width="400" />

In [ ]:
# Create model by [Exponential Smoothing - Holt's] method
# Run for train set
holts.training1 <- holt(holts.training,h=3)
autoplot(forecast(holts.training1))
summary(holts.training1) 

<img src="https://drive.google.com/uc?export=view&id=144ybsh20hbuzjpBQW5MmFl6JyUBV4Fdh" width="400" />
<img src="https://drive.google.com/uc?export=view&id=1476YgQwRLaU_MPeELW5s9pJbL4Z7vYY4" width="400" />


In [ ]:
# Validate the train set with test set 
holts.training1 %>% forecast(h = 3) %>% autoplot() +
  autolayer(holts.test)             
# From the graph, the predicted value from train set is not fit to test set. 

<img src="https://drive.google.com/uc?export=view&id=147r9O7l9kT6BhYS3brPWA1_5gTMO0Tdu" width="400" />

In [ ]:
# Conclude: Model from exponential smoothing by month is weak.  
# Reason: Predicted value is not fit to test set.
```

### 4.2.2.	Exponential Smoothing – Holt’s Winter
The Holt’s Winter Model is being used for data whereby there are trends or seasonality. Considering that we lacked 24 months of data to train the model, we failed to proceed with the modeling and encountered an error whereby the seasonal component could not be estimated in the fitting process.

In [ ]:
```{r Time Series - All Country - Exponential Smoothing - Holts Winter}
# Create model by [Exponential Smoothing - Holt's Winter] method
holts.trainingw <- subset(holts.training, start=length(holts.training)-23)
hwtstotal.issuehw <- hw(holts.trainingw,h=13)
````

<img src="https://drive.google.com/uc?export=view&id=14FuOf8kgHqwhXFlTWLMtEOnPz3ODyLJI" width="650" />
<br><br>

### 4.2.3. Stationarity check for ARIMA and SARIMA
The Autoregressive Integrated Moving Average (ARIMA) model and Seasonal Autoregressive Integrated Moving Average (SARIMA)  will be used as the other model for prediction. The series requires differencing to attain stationarity, according to the Augmented Dickey-Fuller (ADF) Test and the Kwiatkowski-Phillips-Schmidt-Shin (KPSS) Test we have performed has below.

In [ ]:
```{r Time Series - All Country - ARIMA by month}
# Train the time series model by using ARIMA with data by month

# Check the [Stationarity] of time series data
# Augmented Dickey-Fuller Test (ADF)
adf.test(tstotal.issue)

<img src="https://drive.google.com/uc?export=view&id=14Jcd1qqdgSM7Tld0sjl1L27nMN-cUMWs" width="400" />

- HO: Series is Non-Stationary; H1: Series is Stationary
- P-value > 0.05, failed to reject HO; series is non-stationary.


In [ ]:
# Kwiatkowski-Phillips-Schmidt-Shin (KPSS)
kpss.test(tstotal.issue, null="Trend")
```

<img src="https://drive.google.com/uc?export=view&id=14KpQCC2W4U7vcUhjCoJZFRFwQe2G_EEa" width="520" />

 - HO: Series is Stationary; H1: Series is Non-Stationary
 - P-value > 0.05, failed to reject HO; series is stationary

As one of the test has proved the data is non-stationary, differencing is required to be done while training the ARIMA and SARIMA model. 
<br><br>

### 4.2.4. ARIMA
The ARIMA model was trained by using monthly, weekly and daily dataset, as well as different parameters selected from the ACF and PACF plot. However, from many ARIMA models we have trained, all are weak, the predicted value from train set is not fit to the test set. Below is the illustration from the one of the better ARIMA model:

In [ ]:
```{r Time Series - All Country - ARIMA by day}
# Train the time series model by using ARIMA with data by day
# As checked from section above, the dataset is non-stationery, differencing is required
# 1st differencing, check the ACF and PACF plot
d.training %>% diff()%>% ggtsdisplay()

*First differencing:*

<img src="https://drive.google.com/uc?export=view&id=14LETqBwyn8-ppvKCMuZJ9rS5mnG9jKuN" width="400" />

*Note: The data points on the regular plot exhibited stationary mean and stationary variance. However, seasonality behavior can be detected from the Autocorrelation Function (ACF) plot above. The ACF plot had repeating spikes, on a decreasing trend, and followed a regular cycle.*



In [ ]:
# Arima on train set (6,1,0)
dmodel.train = arima(d.training, order=c(6,1,0))
summary(dmodel.train)
checkresiduals(dmodel.train) 

<img src="https://drive.google.com/uc?export=view&id=14Rk_ys-fdzkaaSot7aMOzDzPADKElRtz" width="500" /> 
<img src="https://drive.google.com/uc?export=view&id=14kb0BIynz8QDNo8-o5QEjqV6rcAm-JGX" width="450" />

In [ ]:
# Plot it in test set
dmodel.train %>% forecast(h = 87) %>% autoplot() + autolayer(d.test)
pred.test = Arima(d.test, model = dmodel.train) 
accuracy(pred.test)
# The predicted value from train set is different from test set

# Conclude: Model from ARIMA by day is weak. 
# Reason: Predicted value is not fit to test set.
```

<img src="https://drive.google.com/uc?export=view&id=14maoajOxa6s-sBdCG4Hr_N9I5EIfZYzE" width="450" /> 

<img src="https://drive.google.com/uc?export=view&id=14o0ZbhaViqQaPqRGQD7XEdmI_HP9mJ-z" width="450" />
<br><br>

### 4.2.5. SARIMA

Seasonal Autoregressive Integrated Moving Average (SARIMA) is a more sophisticated model as compared to ARIMA as it allows direct seasonality control by including it as a feature when running the model.






Continuing from Section 4.2.4. where seasonality was observed in the ACF plot, the incorporation of seasonal differencing may eliminate the trend.
<br><br>
<img src="https://drive.google.com/uc?export=view&id=14o9ZiTfzkLTpR19ypGqQLGa7bPmfmuJz" width="400" />

*Graph 12: ACF Plots after First Regular Differencing (from 4.2.4.)*
<br><br>
Observing the ACF plot, there is a cycle repeating every 7 days. This could also be potentially interpreted as changes throughout the week. This cycle then repeats itself at the start of the following week. As a result, seasonal differencing with lag 7 was performed on top of the regular first differencing.
<br><br>
<img src="https://drive.google.com/uc?export=view&id=14rj5SniGkxR9Ru5jHxGVyVg-9OhpOS-G" width="450" />

*Graph 13: Regular, ACF and PACF Plots after First Seasonal Differencing*
<br><br>
After the first seasonal differencing, the seasonality component of the ACF plot was eliminated. The current data had lost its relation to its past value, there is no longer any auto-correlation from its past 7 values. The auto-correlation relation function (PACF) plot reveals the correlation of residuals remaining after removing seasonality and trends present in the residuals.


Illustration from the best SARIMA model: 

In [ ]:
```{r TS All Country - SARIMA by day}
# Arima on train set
# Arima (0,1,3)(0,1,0)7
dmodel.train = arima(d.training, 
                     order=c(0,1,3), 
                     seasonal=list(order=c(0,1,0),period=7))
summary(dmodel.train)
checkresiduals(dmodel.train) 

<img src="https://drive.google.com/uc?export=view&id=14swUbvjqJDWTZtRo5-_KtwVPioGoHe4z" width="500" /> 
<img src="https://drive.google.com/uc?export=view&id=151gEs61x-SoVOrxlmOpNXLKYZLlZAMnE" width="450" />

In [ ]:
# Plot it in test set
dmodel.train %>% forecast(h = 89) %>% autoplot() + autolayer(d.test)
pred.test = Arima(d.test, model = dmodel.train) 
accuracy(pred.test)
# Predicted value from train set is close to test 

# Conclude: Model from SARIMA by day is strong. 
# Reason: Predicted value is closely fit to test set.
```

<img src="https://drive.google.com/uc?export=view&id=157JETD9PABoRuSzPSO29sRxBbCw9531E" width="450" /> 
<img src="https://drive.google.com/uc?export=view&id=15Cc8pInNPg8HLDVizQ8Y0F0qcIe734Jn" width="400" />

The best results were obtained using ARIMA (0,1,3)(0,1,0)7. We had tested multiple models with different Autoregressive (AR) and Moving Average (MA) terms, but none gave a more appropriate model. ARIMA (0,1,3)(0,1,0)7 had AIC of 4,326.85 and p-value of 3.66e-06 for the Ljung-Box Test. On top of that, this model yields the closest fitted value on our test dataset. Hence, ARIMA (0,1,3)(0,1,0)7 was selected as the final model for time series forecasting.
<br><br>

### 4.2.6. Time Series Forecasting – All Country

In [ ]:
```{r TS All Country - forecasting for next 3 months}
# Plotting the prediction for next 3 months based on model trained 
# Arima (0,1,3)(0,1,0)7
dmodel.train %>% forecast(h = 181) %>% autoplot() + autolayer(d.test)
```

<img src="https://drive.google.com/uc?export=view&id=15JK6ORoSeCoNx5VVTcKVdEEbsz4wRVf0" width="450" />

*Graph 14: Prediction Plot for SARIMA – Daily Basis*

Graph 14 shows the number of issues forecasted in the next 3 months for all countries. ARIMA (0,1,3)(0,1,0)7 was used for the forecasting on a daily basis. The line in blue represents the forecasted number of issues and the lighter blue area represents the forecasted range of confidence at 80% (darker blue) and 90% (lighter blue).

As the plot was on a daily basis and, from a resource planning perspective, it is difficult to take reference from a daily chart. Hence, the forecasted value, shown in Graph 16, was put together to reflect as a monthly figure. The final prediction had shown an increasing trend with a slight drop in June 2021.
<br><br>
<img src="https://drive.google.com/uc?export=view&id=15PMtB1Nh3VpOjNA7GMkUgrp77_2FDvwN" width="450" />

*Table 15: Predicted Value for May to July 2021*
<br><br>
<img src="https://drive.google.com/uc?export=view&id=15PVM4JrM9qf6Fd0RECHDWt-qaPwC_2d7" width="600" />

*Graph 16: Prediction of Number of Issues Raised in All Countries from May to July 2021*
<br><br>


## 4.3.	Time Series Modeling – China

Similar modeling was created for number of issue raised in China, including time series regression, Holt's, Holt's Winter, ARIMA and SARIMA, comparison of the best model from each modeling method is in Appendix 3 and Appendix 4. 
<br><br>

### 4.3.1 SARIMA - China
The modeling result of China is not far away from the results of all countries, SARIMA gives the strongest model as compared to the other modeling method. 
<br><br>
Illustration from the best SARIMA model:


In [ ]:
```{r TS China - SARIMA by day}
# Arima on train set
# Arima (0,1,3)(0,1,0)7   
dmodel.train = arima(d.training, 
                     order=c(0,1,3), 
                     seasonal=list(order=c(0,1,0),period=7))
summary(dmodel.train)
checkresiduals(dmodel.train) 


<img src="https://drive.google.com/uc?export=view&id=15UAF5v9EgCv9GtcYT2KlJQ54kJ99MoM3" width="445" /> 
<img src="https://drive.google.com/uc?export=view&id=15_XuXSATKsr74A_rKofvKhTac9_ej1__" width="450" />

In [ ]:
# plot to test set
dmodel.train %>% forecast(h = 89) %>% autoplot() + autolayer(d.test)
pred.test = Arima(d.test, model = dmodel.train) 
accuracy(pred.test)
# Predicted value from train set is closely fit to test set 

# Conclude: Model from ARIMA by day is good. 
# Reason: Predicted value is closely fit to test set.
```

<img src="https://drive.google.com/uc?export=view&id=15djsvLJI5FHyR2EHyPH0F5Xog-cosiSm" width="450" /> 

<img src="https://drive.google.com/uc?export=view&id=15sXfjnLU5_EglNKKyHnK8FnWM8jXMuPC" width="450" />
<br><br>


The best results were obtained using ARIMA (0,1,3)(0,1,0)7. We had tested multiple models with different AR and MA terms, but none gave a more appropriate model. ARIMA (0,1,3)(0,1,0)7 had AIC of 3,780.49 and p-value of 1.259e-13 for the Ljung-Box Test. On top of that, this model yields the closest fitted value on our test dataset. Hence, ARIMA (0,1,3)(0,1,0)7 was selected as the final model for time series forecasting for China.
<br><br>

### 4.3.2 Time Series Forecasting - China

Forecasting of the number of issues for the next 3 months for China results in Graph 17. The best trained model, ARIMA (0,1,3)(0,1,0)7, was used for the forecasting on a daily basis. The line in blue represents the forecasted number of issues and the lighter blue area represents the forecasted range of confidence at 80% (darker blue) and 90% (lighter blue). 

In [ ]:
```{r TS China - forecasting for next 3 months}
# Plotting the prediction for next 3 months based on model trained
# Arima (0,1,3)(0,1,0)7 
dmodel.train %>% forecast(h = 181) %>% autoplot() + autolayer(d.test)
```

<img src="https://drive.google.com/uc?export=view&id=15wd2fYlVhyefv44t-FS4yQZ0DCo8Slpc" width="450" />

*Graph 17: Prediction Plot for SARIMA – Daily Basis*

As the plot was on a daily basis and, from a resource planning perspective, it is difficult to take reference from a daily chart. Hence, the forecasted value was put together on a monthly figure shown in Graph 18. The final prediction had shown a decreasing trend in the following 3 months. 

<img src="https://drive.google.com/uc?export=view&id=15wtw4DT26X-7evPQD---E44-IpR30dG4" width="600" />

*Graph 18: Prediction of Number of Issues Raised in China from May to July 2021*
<br><br>
<img src="https://drive.google.com/uc?export=view&id=16CnTFeHFPHaejPQcYiYdzJTb1Wxw6Xhr" width="450" />

*Graph 19: Prediction of Number of Issues Raised in China from May to July 2021*
<br><br>



# Appendix
*Appendix 1: Table of Comparison of Time Series Prediction Model of Monthly Issue of All Countries.*
*<img src="https://drive.google.com/uc?export=view&id=13zzrUhiEuUt8zduDf4lusbBQ2tM6AlFP" width="600" />
<br><br>
*Appendix 2: Charts of Comparison of Time Series Prediction Model of Monthly Issue of All Countries.*
<img src="https://drive.google.com/uc?export=view&id=142lkyl5K4RlsUT6l5SuGzmEXH0x409ol" width="600" />
<br><br>
*Appendix 3: Table of Comparison of Time Series Prediction Model of Monthly Issue of China.*
<img src="https://drive.google.com/uc?export=view&id=16ENpGWN8sGczHEuS6sEUb2iXeQkkHskD" width="600" />
<br><br>
*Appendix 4: Charts of Comparison of Time Series Prediction Model of Monthly Issue of China.*
<img src="https://drive.google.com/uc?export=view&id=16INwL4TgJtBNWv49InlNX6a0d9n2VRjX" width="600" />
